# RAGAS 
RAGAS เมทริกซ์จะประกอบไปด้วย 4 metrics เฉพาะ 
- RAGASAnswerRelevancyMetric
- RAGASFaithfulnessMetric
- RAGASContextualPrecisionMetric
- RAGASContextualRecallMetric

RAGAS จะคล้ายกับ deepeval แต่ไม่สามารถให้เหตุผลได้ 

### Required Arguments
- input
- actual_output
- expected_output
- retrieval_context

In [8]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

In [9]:
azure_openai.generate("What the ..")

"I'm an AI language model created to assist and answer questions. How can I help you today?"

In [39]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application
actual_output = "We offer a 30-day full refund at no extra cost."

# Replace this with the expected output from your RAG generator
expected_output = "You are eligible for a 30 day full refund at no extra cost."

# Replace this with the actual retrieved context from your RAG pipeline
retrieval_context = ["All customers are eligible for a 30 day full refund at no extra cost."]

metric = RagasMetric(threshold=0.5, model=azure_model)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output,
    expected_output=expected_output,
    retrieval_context=retrieval_context
)

metric.measure(test_case)
print(metric.score)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

/usr/local/lib/python3.9/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.42, however version 0.21.45 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.9/site-packages/ragas/executor.py", line 96, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/usr/local/lib/python3.9/asyncio/base_events.py", line 647, in run_until_complete
    return future.result()
  File "/usr/local/lib/python3.9/site-packages/ragas/executor.py", line 84, in _aresults
    raise e
  File "/usr/local/lib/python3.9/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
  File "/usr/local/lib/python3.9/asyncio/tasks.py", line 611, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/usr/local/lib/python3.9/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
  File "/usr/local/lib/python3.9/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await 

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

### Error when running RagasMetric, So recommand to run on directly ragas lib modude instance.

### RAGASAnswerRelevancyMetric

In [4]:
# data
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

/usr/local/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for explodinggradients/amnesty_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/explodinggradients/amnesty_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [34]:
from datasets import Dataset, DatasetDict, Features, Value, Sequence

# Define the schema
features = Features({
    'question': Value('string'),
    'ground_truth': Value('string'),
    'answer': Value('string'),
    'contexts': Sequence(Value('string')),
})

# Custom data for the dataset about Thailand
data = [
    {
        "question": "What is the capital of Thailand?",
        "ground_truth": "Bangkok",
        "answer": "Bangkok",
        "contexts": ["Bangkok, the capital of Thailand, is a large city known for its vibrant street life and cultural landmarks."]
    },
    {
        "question": "When is the Songkran festival celebrated in Thailand?",
        "ground_truth": "April 13-15",
        "answer": "April 13-15",
        "contexts": ["Songkran is the Thai New Year festival celebrated annually from April 13th to 15th, known for its water fights and cultural festivities."]
    },
    {
        "question": "What is the official language of Thailand?",
        "ground_truth": "Thai",
        "answer": "Thai",
        "contexts": ["Thai, also known as Siamese, is the official language of Thailand and the native language of the Thai people."]
    },
    {
        "question": "Which famous beach is located in Thailand?",
        "ground_truth": "Phuket",
        "answer": "Phuket",
        "contexts": ["Phuket is a popular tourist destination in Thailand, known for its stunning beaches, clear waters, and vibrant nightlife."]
    },
    {
        "question": "What is the main religion in Thailand?",
        "ground_truth": "Buddhism",
        "answer": "Buddhism",
        "contexts": ["Buddhism is the main religion in Thailand, with about 95 percent of the population identifying as Buddhists."]
    },
    # Add more data as needed
]

# Create the dataset
custom_dataset = Dataset.from_dict({"question": [d["question"] for d in data],
                                    "ground_truth": [d["ground_truth"] for d in data],
                                    "answer": [d["answer"] for d in data],
                                    "contexts": [d["contexts"] for d in data]},
                                   features=features)

# If you need a DatasetDict, you can create it and add your dataset under the appropriate split
custom_dataset_dict = DatasetDict({"eval": custom_dataset})

# Display the dataset
print(custom_dataset_dict)


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 5
    })
})


In [35]:
import os
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY

azure_configs = {
    "base_url": settings.AZURE_OPENAI_ENDPOINT,
    "model_deployment": None,
    "model_name": "gpt-35-turbo",
    "embedding_deployment": None,
    "embedding_name": "text-embedding-ada-002",  # most likely
}

In [36]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model = AzureChatOpenAI(
    openai_api_version=settings.OPENAI_API_VERSION,
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["model_deployment"],
    model=azure_configs["model_name"],
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_version=settings.OPENAI_API_VERSION,
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["embedding_deployment"],
    model=azure_configs["embedding_name"],
)

In [37]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

result = evaluate(
    custom_dataset_dict["eval"], metrics=metrics, llm=azure_model, embeddings=azure_embeddings
)

result

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


{'faithfulness': 1.0000, 'answer_relevancy': 0.9727, 'context_recall': 0.8000, 'context_precision': 1.0000, 'harmfulness': 0.0000}

In [38]:
df = result.to_pandas()
df.head()

,question,ground_truth,answer,contexts,faithfulness,answer_relevancy,context_recall,context_precision,harmfulness
0,What is the capital of Thailand?,Bangkok,Bangkok,"[Bangkok, the capital of Thailand, is a large ...",NaN,0.999999,1.0,1.0,0
1,When is the Songkran festival celebrated in Th...,April 13-15,April 13-15,[Songkran is the Thai New Year festival celebr...,1.0,0.987328,1.0,1.0,0
2,What is the official language of Thailand?,Thai,Thai,"[Thai, also known as Siamese, is the official ...",NaN,0.999998,1.0,1.0,0
3,Which famous beach is located in Thailand?,Phuket,Phuket,[Phuket is a popular tourist destination in Th...,NaN,0.876384,0.0,1.0,0
4,What is the main religion in Thailand?,Buddhism,Buddhism,"[Buddhism is the main religion in Thailand, wi...",NaN,0.999999,1.0,1.0,0


https://docs.ragas.io/en/stable/howtos/customisations/azure-openai.html